In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import linalg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import nltk
import pickle
from nltk import pos_tag
from scipy.sparse import hstack, vstack
from scipy import spatial
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import lightgbm as lgb
from sklearn import linear_model
import gc
# 显示cell运行时长
%load_ext klab-autotime

The klab-autotime extension is already loaded. To reload it, use:
  %reload_ext klab-autotime
time: 4.21 ms


In [4]:
train_counting_path = "/home/kesci/work/counting_feature/train/"
valid_counting_path = "/home/kesci/work/counting_feature/valid/"
test_counting_path = "/home/kesci/work/counting_feature/test/"
train_sim_path = "/home/kesci/work/similarity_feature/train/"
valid_sim_path = "/home/kesci/work/similarity_feature/valid/"
test_sim_path = "/home/kesci/work/similarity_feature/test/"

#统计特征
counting_train_feature_1 = train_counting_path + "basic_feature.csv"
counting_train_feature_2 = train_counting_path + "postation.csv"
counting_valid_feature_1 = valid_counting_path + "basic_feature.csv"
counting_valid_feature_2 = valid_counting_path + "postation.csv"
counting_test_feature_1 = test_counting_path + "basic_feature.csv"
counting_test_feature_2 = test_counting_path + "postation.csv"
# query,title点击量特征
click_train_feature_1 = train_counting_path + "click_num_q.csv" 
click_train_feature_2 = train_counting_path + "click_num_t.csv"
click_valid_feature_1 = valid_counting_path + "click_num_q.csv"
click_valid_feature_2 = valid_counting_path + "click_num_t.csv"
click_test_feature_1 = test_counting_path + "click_num_q.csv"
click_test_feature_2 = test_counting_path + "click_num_t.csv"
# 点击量特征汇总 06.15
click_train_feature = train_counting_path + "click_num_8dim.csv" 
click_valid_feature = valid_counting_path + "click_num_8dim.csv"
click_test_feature = test_counting_path + "click_num_8dim.csv"
# vector 相似度特征
sim_train_feature_1 = train_sim_path + "consine_sentence_sim.csv"
sim_train_feature_2 = train_sim_path + "consine_word_sim.csv"
sim_valid_feature_1 = valid_sim_path + "consine_sentence_sim.csv"
sim_valid_feature_2 = valid_sim_path + "consine_word_sim.csv"
sim_test_feature_1 = test_sim_path + "consine_sentence_sim.csv"
sim_test_feature_2 = test_sim_path + "consine_word_sim.csv"
# sentence vector特征
sentence_train_feature= train_sim_path + "sentence_vector.csv"
sentence_valid_feature = valid_sim_path + "sentence_vector.csv"
sentence_test_feature = test_sim_path + "sentence_vector.csv"


train_data = "/home/kesci/input/bytedance/first-round/train.csv"
test_data = "/home/kesci/input/bytedance/first-round/test.csv"
model_path = "/home/kesci/work/GBDT/"
colnames = ["query_id","query","query_title_id","title","label"]

time: 3.4 ms


In [6]:
samples = 100000000
chunksize =  5000000
word_dim = 128
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',         #binary=logistic; regression=mse
    'metric': { 'auc', 'binary_logloss'},
    'num_leaves': 30, 
    
    'learning_rate': 0.1,  # 每个booster设置相同的学习率
    'gamma': 0.1,  # 对叶节点个数的惩罚系数
    'is_training_metric':True,
    
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    # 'verbose': 1
}

time: 2.74 ms


In [4]:
########### 后500w数据 训练GBDT ##############
########### 无样本weight 
########### 后500w数据 训练GBDT ##############
########### 无样本weight 
########### 后500w数据 训练GBDT ##############
########### 无样本weight 

# 6+5+ 22+10 = 43dim

time: 575 µs


In [7]:
################################################# 0. label
skip_num = int(samples/chunksize) - 1
train_label = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*skip_num,nrows=chunksize,lineterminator="\n")["label"]
skip_num = int(samples/chunksize) - 2
nrows_num=1000000
valid_label = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*skip_num,nrows=nrows_num,lineterminator="\n")["label"]

print(train_label.shape)
print(valid_label.shape)
################################################## 0. get sample_weight
# def func(x):
#     default = 1
#     if x == 0:
#         default = 0.5
#     return default
# train_label.to_frame()["label"].apply(func=func)
train_weight = train_label.to_frame()["label"].map({0:0.5,1:1})
valid_weight = valid_label.to_frame()["label"].map({0:0.5,1:1})
print(train_weight.shape,valid_weight.shape)

(5000000,)
(1000000,)
(5000000,) (1000000,)
time: 1min 4s


In [8]:
################################################# 1. valid_data
nrows_num = 1000000
sim_feature_1 = pd.read_csv(sim_valid_feature_1, header=None,nrows=nrows_num,lineterminator="\n")
sim_feature_2 = pd.read_csv(sim_valid_feature_2, header=None,nrows=nrows_num,lineterminator="\n")
counting_feature_1 = pd.read_csv(counting_valid_feature_1, header=None,nrows=nrows_num,lineterminator="\n")
counting_feature_2 = pd.read_csv(counting_valid_feature_2, header=None,nrows=nrows_num,lineterminator="\n")
sentence_feature = pd.read_csv(sentence_valid_feature, header=None,nrows=nrows_num,lineterminator="\n")
click_feature = pd.read_csv(click_valid_feature, header=None,nrows=nrows_num,lineterminator="\n")
combine_feature = np.hstack((
                         sim_feature_1,
                         sim_feature_2,
                         counting_feature_1,
                         counting_feature_2,
                         click_feature,
                         sentence_feature
                         ))
print("valid: ",combine_feature.shape)

# get lgb data
dvalid_base = lgb.Dataset(combine_feature, 
                        label=valid_label,
                        weight=valid_weight)
del sim_feature_1,sim_feature_2,counting_feature_1,counting_feature_2,combine_feature,sentence_feature
gc.collect()

valid:  (1000000, 179)


19

time: 42.5 s


In [9]:
################################################ 2. train_data
sim_feature_1 = pd.read_csv(sim_train_feature_1, header=None,lineterminator="\n")
sim_feature_2 = pd.read_csv(sim_train_feature_2, header=None,lineterminator="\n")
counting_feature_1 = pd.read_csv(counting_train_feature_1, header=None,lineterminator="\n")
counting_feature_2 = pd.read_csv(counting_train_feature_2, header=None,lineterminator="\n")
sentence_feature = pd.read_csv(sentence_train_feature, header=None,lineterminator="\n")
click_feature = pd.read_csv(click_train_feature, header=None,lineterminator="\n")
combine_feature = np.hstack((
                         sim_feature_1,
                         sim_feature_2,
                         counting_feature_1,
                         counting_feature_2,
                         click_feature,
                         sentence_feature
                         ))
print("train: ",combine_feature.shape)

# get lgb data
dtrain_base = lgb.Dataset(combine_feature, 
                        label=train_label,
                        weight=train_weight)
del sim_feature_1,sim_feature_2,counting_feature_1,counting_feature_2,combine_feature,sentence_feature
gc.collect()

train:  (5000000, 179)


0

time: 3min 9s


In [10]:
################################################ 3. train_model

gbm = lgb.train(params,
                train_set=dtrain_base,
                num_boost_round=250,
                valid_sets=[dvalid_base,dtrain_base],
                valid_names = ["valid","train"],
                # valid_sets=[lgb_eval],
                # valid_names = ["valid"] ,
                verbose_eval=10,
                early_stopping_rounds=15,
                # feval=cal_qauc
               )
pickle.dump(gbm,open(model_path+"lgb_model.pkl","wb"))

# Early stopping, best iteration is:
# [242]	train's binary_logloss: 0.52155	train's auc: 0.695284	0.523901 valid's auc: 0.6926311

# Early stopping, best iteration is:
# [217]	train's auc: 0.699427	train's: 0.614691	valid's auc: 0.695739	binary_logloss: 0.617492

Training until validation scores don't improve for 15 rounds.
[10]	train's auc: 0.693336	train's binary_logloss: 0.62814	valid's auc: 0.692516	valid's binary_logloss: 0.628875
[20]	train's auc: 0.695964	train's binary_logloss: 0.61902	valid's auc: 0.695167	valid's binary_logloss: 0.619891
[30]	train's auc: 0.697792	train's binary_logloss: 0.616414	valid's auc: 0.696876	valid's binary_logloss: 0.617418
[40]	train's auc: 0.699135	train's binary_logloss: 0.615174	valid's auc: 0.698056	valid's binary_logloss: 0.616308
[50]	train's auc: 0.700101	train's binary_logloss: 0.614408	valid's auc: 0.698867	valid's binary_logloss: 0.615653
[60]	train's auc: 0.700788	train's binary_logloss: 0.613886	valid's auc: 0.699389	valid's binary_logloss: 0.615235
[70]	train's auc: 0.701364	train's binary_logloss: 0.61345	valid's auc: 0.699809	valid's binary_logloss: 0.614932
[80]	train's auc: 0.701858	train's binary_logloss: 0.6131	valid's auc: 0.700142	valid's binary_logloss: 0.614688
[90]	train's auc: 0.702

In [11]:
################################################ 4. predict
del dtrain_base,dvalid_base
gc.collect()
sim_feature_1 = pd.read_csv(sim_test_feature_1, header=None,lineterminator="\n")
sim_feature_2 = pd.read_csv(sim_test_feature_2, header=None,lineterminator="\n")
counting_feature_1 = pd.read_csv(counting_test_feature_1, header=None,lineterminator="\n")
counting_feature_2 = pd.read_csv(counting_test_feature_2, header=None,lineterminator="\n")
sentence_feature = pd.read_csv(sentence_test_feature, header=None,lineterminator="\n")
click_feature = pd.read_csv(click_test_feature, header=None,lineterminator="\n")

combine_feature = np.hstack((
                         sim_feature_1,
                         sim_feature_2,
                         counting_feature_1,
                         counting_feature_2,
                         click_feature,
                         sentence_feature
                         ))
print("test: ",combine_feature.shape)

# predict
lgb_model = pickle.load(open(model_path+"lgb_model.pkl","rb"))
predict_ans = lgb_model.predict(combine_feature,ntree_limit=lgb_model.best_iteration)
del sim_feature_1,sim_feature_2,counting_feature_1,counting_feature_2,combine_feature,sentence_feature
gc.collect()

test:  (5000000, 179)


12

time: 6min 9s


In [12]:
print(predict_ans.shape)
print(predict_ans[:10])

(5000000,)
[0.55691499 0.56963687 0.56857211 0.58509736 0.64476613 0.73858848
 0.52743986 0.31526443 0.27143407 0.30951374]
time: 1.37 ms


In [13]:
colnames = ["query_id","query","query_title_id","title"]
test_df = pd.read_csv(test_data,names=colnames, header=None,lineterminator="\n")
test_df["pred"] = predict_ans.reshape(-1)
print(test_df.shape)

save_path = "/home/kesci/submit/"
test_df.to_csv(save_path+"500w_sample_weight_sentence_vector_click_more.csv",columns=["query_id","query_title_id","pred"],header=None,index=None)
del test_df
gc.collect()

(5000000, 5)


7

time: 38 s


In [14]:
!cat /home/kesci/submit/500w_sample_weight_sentence_vector_click_more.csv | wc -l
!cat /home/kesci/submit/500w_sample_weight_sentence_vector_click_more.csv | head -n 5

5000000
1,3,0.5569149929700422
1,1,0.5696368738374294
1,4,0.5685721103342345
1,2,0.5850973625985558
2,1,0.6447661341325742
cat: write error: Broken pipe
time: 1.4 s


In [15]:
!./kesci_submit -token f3ab22d4d5cb5ea4 -file /home/kesci/submit/500w_sample_weight_sentence_vector_click_more.csv

Kesci Submit Tool
Result File: /home/kesci/submit/500w_sample_weight_sentence_vector_click_more.csv (134.57 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d04cf4fb52341002b806850","Competition":"5cc51043f71088002c5b8840","Team":"5cf39c212e1a1d002b3916af","UploadDate":"2019-06-15T10:58:23.148Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.82.108.66","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"1560596290256dc6b93.csv","ResultFileRealName":"500w_sample_weight_sentence_vector_click_more.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}

time: 14.4 s


In [16]:
import matplotlib.pyplot as plt
import lightgbm as lgb
model_path = "/home/kesci/work/GBDT/"
lgb_model = pickle.load(open(model_path+"lgb_model.pkl","rb"))
plt.figure(figsize=(12,6))
lgb.plot_importance(lgb_model, max_num_features=20)
plt.title("Feature importances")
plt.show()

# import matplotlib.pyplot as plt
# model_path = "/home/kesci/work/GBDT/train/"
# lgb_model = pickle.load(open(model_path+"lgb_model.pkl","rb"))
# plt.figure(figsize=(12,6))
# lgb.plot_importance(lgb_model, max_num_features=20)
# plt.title("Feature importances")
# plt.show()

<Figure size 864x432 with 0 Axes>

<Figure size 432x288 with 1 Axes>

time: 759 ms


In [41]:
!cat /home/kesci/work/similarity_feature/train/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/train/sentence_vector.csv | wc -l

!cat /home/kesci/work/similarity_feature/valid/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid/sentence_vector.csv | wc -l

!cat /home/kesci/work/similarity_feature/test/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/test/sentence_vector.csv | wc -l

5000000
5000000
5000000
5000000
5000000
5000000


In [42]:
!cat /home/kesci/work/similarity_feature/train/consine_word_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid/consine_word_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/test/consine_word_sim.csv | wc -l

5000000
5000000
5000000


In [9]:
!cat /home/kesci/input/bytedance/first-round/test.csv |head -n 1


1,11202 184 50256,3,11202 184 2346 2527 274 383 34 1033 156 18 50256 19 27 11533 7761 1600 2769	
cat: write error: Broken pipe
time: 555 ms
